# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Naive-Bayes" data-toc-modified-id="Naive-Bayes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Naive Bayes</a></div><div class="lev2 toc-item"><a href="#分析" data-toc-modified-id="分析-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>分析</a></div><div class="lev3 toc-item"><a href="#训练集" data-toc-modified-id="训练集-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>训练集</a></div><div class="lev3 toc-item"><a href="#先验" data-toc-modified-id="先验-112"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>先验</a></div><div class="lev3 toc-item"><a href="#测试集" data-toc-modified-id="测试集-113"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>测试集</a></div><div class="lev4 toc-item"><a href="#pos_test" data-toc-modified-id="pos_test-1131"><span class="toc-item-num">1.1.3.1&nbsp;&nbsp;</span>pos_test</a></div><div class="lev4 toc-item"><a href="#neg_test" data-toc-modified-id="neg_test-1132"><span class="toc-item-num">1.1.3.2&nbsp;&nbsp;</span>neg_test</a></div><div class="lev3 toc-item"><a href="#相关概念" data-toc-modified-id="相关概念-114"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>相关概念</a></div><div class="lev2 toc-item"><a href="#MagicCheng" data-toc-modified-id="MagicCheng-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>MagicCheng</a></div><div class="lev1 toc-item"><a href="#梯度下降法" data-toc-modified-id="梯度下降法-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>梯度下降法</a></div><div class="lev2 toc-item"><a href="#梯度下降法推导" data-toc-modified-id="梯度下降法推导-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>梯度下降法推导</a></div><div class="lev3 toc-item"><a href="#主要观点" data-toc-modified-id="主要观点-211"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>主要观点</a></div><div class="lev3 toc-item"><a href="#实现步骤" data-toc-modified-id="实现步骤-212"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>实现步骤</a></div><div class="lev2 toc-item"><a href="#网上参考的代码" data-toc-modified-id="网上参考的代码-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>网上参考的代码</a></div>

In [2]:
ls

neg_test.txt   pos_test.txt   w2_code.ipynb  w2_task.ipynb
neg_train.txt  pos_train.txt  w2_note.ipynb  w2_task_nice_sunoonlee.ipynb


# Naive Bayes

自己用 Python 实现 Naive Bayes 方法，并在给定的数据集上验证效果

In [12]:
import jieba

In [33]:
import jieba.posseg as pseg

In [13]:
from collections import Counter

In [79]:
# 测试代码和数据
lm = Counter()
line_no = 0
limit = 1000

for line in open("neg_train.txt","r"):
    line_no += 1
    if line_no > limit:
        break
    words = pseg.cut(line.strip())
    
    for word, flag in words:
        if flag == 'x' or flag == 'uj' or flag == 'ul':
            continue
        lm[(word.encode('utf-8'), flag.encode('utf-8'))] += 1

In [82]:
for (word, flag),cnt in lm.most_common(10):
    print('%s %s %d' %(word, flag,cnt))

我 r 650
是 v 584
不 d 378
酒店 n 351
没有 v 301
就 d 287
在 p 272
也 d 266
房间 n 264
有 v 241


## 分析

### 训练集

In [773]:
lmp = Counter()
lmn = Counter()

for linep in open("pos_train.txt", "r"):
    wordsp = pseg.cut(linep.strip())
    
    for word, flag in wordsp:
        if flag == 'x' or flag == 'uj' or flag == 'ul':
            continue
        lmp[(word.encode('utf-8'), flag.encode('utf-8'))] += 1


for linen in open("neg_train.txt", "r"):
    wordsn = pseg.cut(linen.strip())

    for word, flag in wordsn:
        if flag == 'x' or flag == 'uj' or flag == 'ul':
            continue
        lmn[(word.encode('utf-8'), flag.encode('utf-8'))] += 1

In [774]:
sp = float(sum(lmp.values()))
for key, cnt in lmp.items():
    lmp[key] = (lmp[key]+1.0) / (sp + len(lmp))

In [775]:
sp, len(lmp)

(460008.0, 28273)

In [776]:
sn = float(sum(lmn.values()))
for key, cnt in lmn.items():
    lmn[key] = (lmn[key]+1.0) / (sn + len(lmn))

In [777]:
sn, len(lmn)

(410530.0, 22197)

In [778]:
for (word, flag),cnt in lmp.most_common(10):
    print('%s %s %f' %(word, flag,cnt))

是 v 0.018217
我 r 0.015106
也 d 0.010666
在 p 0.009339
有 v 0.008956
不错 a 0.008086
很 d 0.007844
都 d 0.007827
这 r 0.007182
很 zg 0.006849


In [779]:
for (word, flag),cnt in lmn.most_common(10):
    print('%s %s %f' %(word, flag,cnt))

是 v 0.018289
我 r 0.017568
不 d 0.011913
酒店 n 0.010471
也 d 0.008398
没有 v 0.008375
在 p 0.008148
就 d 0.007961
房间 n 0.007499
有 v 0.007282


In [780]:
traindict = {}

traindict = {"1":lmp, "0":lmn}

In [781]:
traindict['0'][('\xe6\x8a\xb1\xe6\x80\xa8', 'v')]

4.3907590698061365e-05

### 先验

In [593]:
npos = len(open("pos_train.txt", "r").readlines())

In [594]:
nneg = len(open("neg_train.txt", "r").readlines())

In [595]:
ppos = float(npos)/float(npos+nneg)
ppos

0.47112177662084115

In [596]:
pneg = 1 - ppos
pneg

0.5288782233791589

### 测试集

In [782]:
resultp = []

f = open('pos_test.txt', 'r')
for line in open("pos_test.txt"):
    line = f.readline()
    resultp.append(line)

In [783]:
len(resultp)

4965

In [784]:
resultn = []

f = open('neg_test.txt', 'r')
for line in open("neg_test.txt"):
    line = f.readline()
    resultn.append(line)

In [785]:
len(resultn)

5573

In [798]:
def cutSen(sen):
    sw = []
    s = pseg.cut(sen.strip())
    for word, flag in s:
        if flag == 'x' or flag == 'uj' or flag == 'ul':
            continue
        sw.append((word.encode('utf-8'), flag.encode('utf-8')))
    return sw

In [799]:
def rateWord(wf):
    
    wfs = cutSen(wf)
    
    import math
    rateps = 0
    ratens = 0
    for i in range(len(wfs)):
        wfp = traindict["1"][wfs[i]]
        wfn = traindict["0"][wfs[i]]
        
        if wfp == 0:
            ratep = math.log(1.0 / (sp + len(lmp) + 1.0))
        else:
            ratep = math.log(wfp)
        
        rateps += ratep
        
        if wfn == 0:
            raten = math.log(1.0/ (sn + len(lmn) +1.0))
        else:
            raten = math.log(wfn)
        
        ratens += raten
    
    return rateps + math.log(ppos), ratens + math.log(pneg)

#### pos_test

In [800]:
testResultp = []

for i in range(len(resultp)):
    testResultp.append(rateWord(resultp[i]))

In [801]:
len(testResultp)

4965

In [802]:
n = 0
for i in range(len(testResultp)):
    if testResultp[i][0] > testResultp[i][1]:
        n += 1

In [803]:
n

3808

In [804]:
3808.0/4965

0.766968781470292

#### neg_test

In [805]:
testResultn = []

for i in range(len(resultn)):
    testResultn.append(rateWord(resultn[i]))

In [806]:
len(testResultn)

5573

In [807]:
n = 0
for i in range(len(testResultn)):
    if testResultn[i][0] < testResultn[i][1]:
        n += 1

In [808]:
n

4937

In [809]:
4937.0/5573

0.8858783420061008

### 相关概念

$P(y|S) = \frac{P(y,S)}{P(S)} = \frac{P(S|y)*P(y)}{P(S)}$  

$P(S|y=emotion) = P(w_1,w_2,...,w_n|y=emotion) \approx P(w_1|y=emotion)...P(w_n|y=emotion)$ 假设词之间独立

## MagicCheng

In [1]:
from collections import Counter, defaultdict
from math import log
import jieba

In [2]:
# 先验概率 = 正面评价（负面评价）／所有评价
def readfile(file_path):
    with open(file_path.encode('utf-8')) as f:
        lines = f.readlines()
    return [line.strip() for line in lines]
pos_count = float(len(readfile('./pos_train.txt'))) #不加float结果都是0.0
neg_count = float(len(readfile('./neg_train.txt')))
sums = pos_count + neg_count
prior = [(pos_count / sums), (neg_count / sums)]
prior

[0.47112177662084115, 0.5288782233791589]

In [4]:
def model(train_file):
    '''处理下文件，变成list'''
    lm = Counter()
    for line in readfile(train_file):
        for word in jieba.cut(line):
            lm[word] += 1
    return lm

s = model('./pos_train.txt') + model('./neg_train.txt')
wordlist = set()
for word in s.items():
    wordlist.add(word) #合并文件

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.332 seconds.
Prefix dict has been built succesfully.


In [6]:
def get(train_file):
    """这里用了平滑处理"""
    v_count = len(wordlist)
    goal = model(train_file)
    sum_ = float(sum(goal.values()))
    get = Counter()
    for word, cnt in goal.items():
        get[word] = (cnt + 1) / (sum_ + v_count + 1)
    get = defaultdict(lambda: 1/(sum_ + v_count + 1), get)
    return get

In [7]:
pos_train_lm = get('./pos_train.txt')
neg_train_lm = get('./neg_train.txt')

In [8]:
def classify(content, pos_train_lm, neg_train_lm):
    words = jieba.cut(content)
    result = log(prior[0]/prior[1]) #这里看同学的作业，感觉他的有问题我改了下。
    for word in words:
        result += log(pos_train_lm[word] / neg_train_lm[word])
    return result

In [10]:
s_text = ['./neg_test.txt', './pos_test.txt']
r = [[0,0],[0,0]]
for i in (0,1):
    for line in open(s_text[i]):
        if classify(line,pos_train_lm, neg_train_lm) < 0: # 比较大小过程。
            r[i][0] += 1
        else:
            r[i][1] += 1
total = r[0][0] + r[0][1] + r[1][0] + r[1][1]

In [11]:
confusion_matrix = ('\n'.join(['{:^15}' * 3] * 3)).format(
    'n=' + str(total), 'predicted:neg', 'predicted:pos',
    'actual:neg', r[0][0], r[1][0],
    'actual:pos', r[0][1], r[1][1])
print(confusion_matrix)

    n=10538     predicted:neg  predicted:pos 
  actual:neg        4889           1122      
  actual:pos         684           3843      


In [12]:
true_pos_rate = float(r[1][1])/float(r[1][0] + r[1][1])
true_pos_rate

0.7740181268882175

In [13]:
true_neg_rate = float(r[0][0])/float(r[0][0] + r[0][1])
true_neg_rate

0.8772653866858066

# 梯度下降法


通过梯度下降方法，自己实现一种通用的给定数据找到 $y = wx + b$ 中最优 w 和 b 的程序，并用加噪音数据验证效果，和情感分析的关系？

## 梯度下降法推导

### 主要观点

- 梯度下降法主要目的是找出最优的 w 和 b，基本思想是根据给定的 x 和 y 值，不断更新 w 和 b，将求得的 y 与给定的 y 值进行比较，选择离差平方和最小的 w 和 b

### 实现步骤

- 设函数为：$$ h_\theta(x) = \theta_0 + \theta_1*x $$  
  写成矩阵形式（非常重要）：$$ h(x) = \sum_{i=0}^n \theta_ix_i $$
- 假设给定的 x', y' 分别为给定的值，求离差平方和为：$$ J(\theta_0, \theta_1) = \frac{1}{2m}\sum_{i=1}^m(h_\theta(x')-y')^2 $$
  乘以 1/2m 方便求导后化简。注意：这里的 m 表示 (x', y') 的个数，上式的 n 表示 $\theta$ 的个数

- 要求最小值，对离差平方和进行求导：$$ \frac{\partial}{\partial \theta_j} J(\theta_0, \theta_1) = \frac{1}{2m} \sum_{i=1}^m  \frac{\partial}{\partial \theta_j} (h_\theta(x')-y')^2 $$

j = 1 时：$$ \frac{\partial}{\partial \theta_j} J(\theta_0, \theta_1) = \frac{1}{m} \sum_{i=1}^m  (h_\theta(x')-y') * x' $$

j = 0 时：$$ \frac{\partial}{\partial \theta_j} J(\theta_0, \theta_1) = \frac{1}{m} \sum_{i=1}^m  (h_\theta(x')-y') $$

算法： 

Repeat {

  $$ \theta_0: = \theta_0 - \alpha \frac{1}{m} \sum_{i=1}^m  (h_\theta(x')-y') $$
  $$ \theta_1: = \theta_1 - \alpha \frac{1}{m} \sum_{i=1}^m  (h_\theta(x')-y')*x' $$
}

In [1]:
def g(x):
    
    return w*x + b

In [4]:
import numpy as np

In [44]:
def gradient(X, y, rate, eps, times):
    
    import numpy as np
    m = len(X)
    n = X.shape[1]   
    theta = np.array(np.zeros(n)).reshape(n,1)
    #x_train = np.column_stack((np.ones(m), x))
    x_train = X
    y_train = y.reshape(m, 1)
    Jcosts = []
    Jcost = 0
    Jcostt = 0

    for i in range(times):
        theta -= rate * ((x_train.dot(theta) - y_train) * x_train).sum(axis=0).reshape(n,1)
        Jcost = sum((x_train.dot(theta) - y_train) ** 2)/(2*m)
        Jcosts.append(Jcost)
        
        # 新的 cost 大于上一个时，说明步子太大，缩小步长
        if Jcost > Jcostt:
            rate /= 10.0
        # 梯度下降!
        # 也可以直接 if Jcost < eps，但是不能避免 Jcost 永远无法收敛到 eps 大小的情况，
        # 所以需要找一个中间变量，用来做差，用 Jcost 的差作为判断循环停止的标准
        if abs(Jcost - Jcostt) < eps: 
            break
        #else:
        # 这里的 else 容易让人误会，可以不要，每次循环结束时，直接把 Jcost 赋给 Jcostt
        Jcostt = Jcost
    return theta, Jcosts

In [4]:
import numpy as np

In [18]:
Y = X[:,0] + 2*X[:,1] + np.random.random()

In [5]:
X = np.column_stack((np.ones(50), np.linspace(-10,10)))

In [59]:
gradient(X,Y,0.001,0.00001,100)[0]

array([[ 0.7313814 ],
       [ 2.00000001]])

In [51]:
x = np.array([(1, 0., 3),(1, 1., 3),(1, 2., 3),(1, 3., 2), (1, 4, 4)])
y = np.array([95.364, 97.217205, 75.195834, 60.105519, 49.342380])
y = y.reshape(5,1)

In [52]:
x

array([[ 1.,  0.,  3.],
       [ 1.,  1.,  3.],
       [ 1.,  2.,  3.],
       [ 1.,  3.,  2.],
       [ 1.,  4.,  4.]])

In [53]:
y

array([[ 95.364   ],
       [ 97.217205],
       [ 75.195834],
       [ 60.105519],
       [ 49.34238 ]])

In [57]:
gradient(x,y,0.01,0.0001,10000)[0] # 实际 2277 次即收敛

array([[ 85.16466796],
       [-12.93716954],
       [  5.21874193]])

## 网上参考的代码

In [7]:
x = np.array([(1, 0., 3),(1, 1., 3),(1, 2., 3),(1, 3., 2), (1, 4, 4)])
y = np.array([95.364, 97.217205, 75.195834, 60.105519, 49.342380])
y = y.reshape(5,1)

epsilon = 0.0001

# 学习率
alpha = 0.01
diff = [0, 0]
max_itor = 1000
error1 = 0
error0 = 0
cnt = 0
m = len(x)
errors = []

# 初始化参数
theta = np.array([0, 0, 0]).reshape(3,1)

while True:
    cnt += 1

    # 参数迭代计算
    for i in range(m):
        theta = theta - alpha * ((x.dot(theta) - y) * x).sum(axis=0).reshape(3,1)
    # 计算损失函数
    #error1 = 0
    #for lp in range(len(x)):
    error1 += sum((x.dot(theta) - y) ** 2)/(2*m)
    errors.append(error1)

    if abs(error1-errors[-1]) < epsilon:
        break
#    else:
#        error0 = error1

    print ' theta0 : %f, theta1 : %f, theta2 : %f, error1 : %f' % (theta[0], theta[1], theta[2], errors[-1])
#print 'Done: theta0 : %f, theta1 : %f, theta2 : %f' % (theta0, theta1, theta2)
#print '迭代次数: %d' % cnt